In [1]:
# basics
import intake
import xarray as xr
import dask  # memory-efficient parallel computation and delayed execution (lazy evaluation).
import subprocess as sp

%run gem_helpers.ipynb

uid = getpass.getuser()
image_path = make_tempdir("intake_demo_plots")
data_cache_path = make_tempdir("intake_demo_data")

# this function is also part of the gem helpers
def get_list_from_cat(catalog, column):
    """A helper function for getting the contents of a column in an intake catalog.

    Call with the catalog to be inspected and the column of interest."""
    return sorted(catalog.unique(column)[column]["values"])

catalog_file = "/work/ka1081/Catalogs/dyamond-nextgems.json"
cat = intake.open_esm_datastore(catalog_file)

#cat

#var = "tas"
var = "sst" 
hits = cat.search(simulation_id=["HQYS"], variable_id=[var],frequency="1month")

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)


In [3]:
#get_from_cat(hits,["grid_label","grid_id","simulation_id", "variable_id"])

file_cat = {}
#for simulation_id in ("ngc2009", "ngc2012"):
for simulation_id in ("HQYS"):
    file_cat[simulation_id] = get_list_from_cat(
        hits.search(simulation_id=simulation_id), "uri"
    )

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = df[columns].apply(_find_unique, result_type='reduce').to_dict()


ValueError: Length of values (0) does not match length of index (1)

In [4]:
    
outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_timmean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-timmean",
                "-sellonlatbox,0,360,-30,30",
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = df[columns].apply(_find_unique, result_type='reduce').to_dict()


ValueError: Length of values (0) does not match length of index (1)